In [1]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import warnings
import seaborn as sns 
from scipy import stats
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from scipy.stats.stats import ttest_ind
from scipy import stats

warnings.filterwarnings('ignore')
###sns.set(style="whitegrid")
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'
table_name = 'lifeexpectancy'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

lifeDf = pd.read_sql_query('select * from lifeexpectancy',con=engine)
engine.dispose()

# 1) Detect the problems with the data such as missing values and outliers. Are there any nonsense values that seem to be stemmed from the data collection? For the missing values, discuss which technique would be the most suitable one in filling out these values. Regarding the outliers, discuss their potential effects on your analysis and select an appropriate method to deal with them.

In [2]:
lifeDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
Country                            2938 non-null object
Year                               2938 non-null int64
Status                             2938 non-null object
Life expectancy                    2928 non-null float64
Adult Mortality                    2928 non-null float64
infant deaths                      2938 non-null int64
Alcohol                            2744 non-null float64
percentage expenditure             2938 non-null float64
Hepatitis B                        2385 non-null float64
Measles                            2938 non-null int64
 BMI                               2904 non-null float64
under-five deaths                  2938 non-null int64
Polio                              2919 non-null float64
Total expenditure                  2712 non-null float64
Diphtheria                         2919 non-null float64
 HIV/AIDS                          2938 non-null

In [3]:
lifeDf.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [4]:
for column_name in lifeDf.columns:
    print("Unique values in column {} are: {}".format(column_name, lifeDf[column_name].unique()))

Unique values in column Country are: ['Afghanistan' 'Albania' 'Algeria' "Côte d'Ivoire" 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia (Plurinational State of)'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Brunei Darussalam'
 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon'
 'Canada' 'Central African Republic' 'Chad' 'Chile' 'China' 'Colombia'
 'Comoros' 'Congo' 'Cook Islands' 'Costa Rica' 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Slovakia'
 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Ethiopia' 'Fiji' 'Finland'
 'France' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Honduras'
 'Hungary' 'Iceland' 

In [5]:
lifeDf.nunique()

Country                             193
Year                                 16
Status                                2
Life expectancy                     362
Adult Mortality                     425
infant deaths                       209
Alcohol                            1076
percentage expenditure             2328
Hepatitis B                          87
Measles                             958
 BMI                                608
under-five deaths                   252
Polio                                73
Total expenditure                   818
Diphtheria                           81
 HIV/AIDS                           200
GDP                                2490
Population                         2278
 thinness  1-19 years               200
 thinness 5-9 years                 207
Income composition of resources     625
Schooling                           173
dtype: int64

In [6]:
fill_list = ['Life expectancy', 'Adult Mortality', 'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population', ' thinness 1-19 years', ' Schooling', 'Income composition of resources', 'Schooling']

In [7]:
lifeDf["Hepatitis B"].fillna(lifeDf["Hepatitis B"].mean(), inplace=True)
lifeDf["Adult Mortality"].fillna(lifeDf["Adult Mortality"].mean(), inplace=True)
lifeDf["Alcohol"].fillna(lifeDf["Alcohol"].mean(), inplace=True)
#lifeDf["Measles"].fillna(lifeDf["Measles"].mean(), inplace=True)
lifeDf["Polio"].fillna(lifeDf["Polio"].mean(), inplace=True)
#lifeDf["Diphtheria"].fillna(lifeDf["Diphtheria"].mean(), inplace=True)
#lifeDf["HIV/AIDS"].fillna(lifeDf["HIV/AIDS"].mean(), inplace=True)
lifeDf["Population"].fillna(lifeDf["Population"].mean(), inplace=True)
lifeDf["GDP"].fillna(lifeDf["GDP"].mean(), inplace=True)
lifeDf["Schooling"].fillna(lifeDf["Schooling"].mean(), inplace=True)
lifeDf["Income composition of resources"].fillna(lifeDf["Income composition of resources"].mean(), inplace=True)
#lifeDf["thinness 1-19 years"].fillna(lifeDf["thinness 1-19 years"].mean(), inplace=True)
lifeDf["percentage expenditure"].fillna(lifeDf["percentage expenditure"].mean(), inplace=True)
#lifeDf["under-five deaths"].fillna(lifeDf["under-five deaths"].mean(), inplace=True)
#lifeDf["BMI"].fillna(lifeDf["BMI"].mean(), inplace=True)

## remaining

In [8]:
lifeDf.isnull().sum()*100/lifeDf.isnull().count()

Country                            0.000000
Year                               0.000000
Status                             0.000000
Life expectancy                    0.340368
Adult Mortality                    0.000000
infant deaths                      0.000000
Alcohol                            0.000000
percentage expenditure             0.000000
Hepatitis B                        0.000000
Measles                            0.000000
 BMI                               1.157250
under-five deaths                  0.000000
Polio                              0.000000
Total expenditure                  7.692308
Diphtheria                         0.646698
 HIV/AIDS                          0.000000
GDP                                0.000000
Population                         0.000000
 thinness  1-19 years              1.157250
 thinness 5-9 years                1.157250
Income composition of resources    0.000000
Schooling                          0.000000
dtype: float64

In [9]:
countryList = lifeDf.Country

for i in range(0, len(lifeDf.Country)):
    if pd.isnull(lifeDf.Country[i]):
        lifeDf.Country[i] = lifeDf.Country[i-1]
        
lifeDf["Country"].unique()

array(['Afghanistan', 'Albania', 'Algeria', "Côte d'Ivoire", 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica',
       'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Slovakia', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia

In [10]:
lifeDf.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                      0
infant deaths                        0
Alcohol                              0
percentage expenditure               0
Hepatitis B                          0
Measles                              0
 BMI                                34
under-five deaths                    0
Polio                                0
Total expenditure                  226
Diphtheria                          19
 HIV/AIDS                            0
GDP                                  0
Population                           0
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources      0
Schooling                            0
dtype: int64

In [11]:
lifeDf.columns = lifeDf.columns.str.strip().str.lower().str.replace(' ', '_')

In [12]:
lifeDf.head()

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_expenditure,hepatitis_b,measles,...,polio,total_expenditure,diphtheria,hiv/aids,gdp,population,thinness__1-19_years,thinness_5-9_years,income_composition_of_resources,schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [13]:
lifeDf["life_expectancy"].fillna(lifeDf["life_expectancy"].mean(), inplace=True)
lifeDf["thinness__1-19_years"].fillna(lifeDf["thinness__1-19_years"].mean(), inplace=True)

In [14]:
lifeDf.isnull().sum()

country                              0
year                                 0
status                               0
life_expectancy                      0
adult_mortality                      0
infant_deaths                        0
alcohol                              0
percentage_expenditure               0
hepatitis_b                          0
measles                              0
bmi                                 34
under-five_deaths                    0
polio                                0
total_expenditure                  226
diphtheria                          19
hiv/aids                             0
gdp                                  0
population                           0
thinness__1-19_years                 0
thinness_5-9_years                  34
income_composition_of_resources      0
schooling                            0
dtype: int64

In [18]:
lifeDf.drop(['thinness_5-9_years', 'status'], inplace =True)

KeyError: "['thinness_5-9_years' 'status'] not found in axis"

In [16]:
lifeDf.isnull().sum()

country                              0
year                                 0
status                               0
life_expectancy                      0
adult_mortality                      0
infant_deaths                        0
alcohol                              0
percentage_expenditure               0
hepatitis_b                          0
measles                              0
bmi                                 34
under-five_deaths                    0
polio                                0
total_expenditure                  226
diphtheria                          19
hiv/aids                             0
gdp                                  0
population                           0
thinness__1-19_years                 0
thinness_5-9_years                  34
income_composition_of_resources      0
schooling                            0
dtype: int64

In [17]:
lifeDf.describe()

,year,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_expenditure,hepatitis_b,measles,bmi,under-five_deaths,polio,total_expenditure,diphtheria,hiv/aids,gdp,population,thinness__1-19_years,thinness_5-9_years,income_composition_of_resources,schooling
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2904.000000,2938.000000,2938.000000,2712.00000,2919.000000,2938.000000,2938.000000,2.938000e+03,2938.000000,2904.000000,2938.000000,2938.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.507640,124.080302,117.926501,3.916288,1987.914858,22.586855,11467.272489,20.044034,160.445548,23.352143,2.49832,23.716912,5.077785,13136.800417,5.381546e+07,4.394535,4.508882,0.204820,3.264381
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.200000,74.000000,0.000000,1.092500,4.685343,80.940461,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,580.486996,4.189172e+05,1.600000,1.500000,0.504250,10.300000
50%,2008.000000,72.000000,144.000000,3.000000,4.160000,64.912906,87.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,3116.561755,3.675929e+06,3.400000,3.300000,0.662000,12.100000
75%,2012.000000,75.600000,227.000000,22.000000,7.390000,441.534144,96.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,7483.158469,1.275338e+07,7.100000,7.200000,0.772000,14.100000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000
